In [2]:
#begin with imports
import os
import requests
import arcpy

#set up arcpro project, map, and coordinate system
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]
ref = arcpy.SpatialReference(4326) #using WGS 1983

In [3]:
#set up output path where everything is saved
outputPath = r"C:\ArcGIS\Projects\Lab2_Part1\PRISM"

#but i only want to make it if it does not already exist
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
    print("Created output directory: ", outputPath)
else:
    print("Output directory is already assigned to: ", outputPath)

Output directory is already assigned to:  C:\ArcGIS\Projects\Lab2_Part1


In [4]:
#the url i will get my data from
url = 'https://services.nacse.org/prism/data/public/normals/4km/ppt/14'
#where i will save my url to
urlPath = os.path.join(outputPath, '30yr_normal_4kmM4')

#download the .las file if it isn't already downloaded using requests.get
if not os.path.exists(urlPath):
    response = requests.get(url)
    if response.status_code == 200: #if it downloads successfully (HTTP code 200)
        with open(urlPath, 'wb') as zip_file:
            zip_file.write(response.content)
        print("Downloaded .ZIP file to: ", urlPath)
    else: #if it does not download
        print("Failed to download .ZIP file. Status code: ", response.status_code)
else:
    print(".ZIP already downloaded to: ", urlPath)

.LAS already downloaded to:  C:\ArcGIS\Projects\Lab2_Part1\4342_12_06.las


In [5]:
#must unzip the file and add data to my map
#where i will save the .bil
bilPath = os.path.join(outputPath, '30yr_normal_4kmM4.bil')

#create .bil if it hasn't already been created
if not os.path.exists(bilPath):
    arcpy.management.CreateLasDataset(urlPath, bilPath, spatial_reference=ref)
    print("Converted to LASD: ", bilPath)
else:
    print("LASD already exists at: ", bilPath)

LASD already exists at:  C:\ArcGIS\Projects\Lab2_Part1\4342_12_06.lasd


In [6]:
#converting .lasd to DEM
#it wasn't working for me until i cleared the environment
arcpy.env.workspace = None
arcpy.env.overwriteOutput = True

#where i will save my DEM
demPath = os.path.join(outputPath, 'DEM_4342_12_06')

#create DEM if it hasn't already been created
if not os.path.exists(demPath):
    arcpy.env.workspace = outputPath  #reset environment workspace
    arcpy.conversion.LasDatasetToRaster( #parameters from esri techical support
        lasdPath, demPath, 
        value_field="ELEVATION", 
        interpolation_type="BINNING AVERAGE NATURAL_NEIGHBOR", 
        data_type="FLOAT", 
        sampling_type="CELLSIZE", 
        sampling_value=2
    )
    if os.path.exists(demPath): #success
        print("Created DEM: ", demPath)
    else: #failure
        print("DEM creation failed.")
else:
    print("DEM file already exists at: ", demPath)

DEM file already exists at:  C:\ArcGIS\Projects\Lab2_Part1\DEM_4342_12_06.tif


In [7]:
#converting .lasd to TIN
#where i will save my TIN
tinPath = os.path.join(outputPath, 'TIN_4342_12_06')

#create TIN if it hasn't already been created using try/except loop for fun
if not os.path.exists(tinPath):
    try: #success
        arcpy.ddd.LasDatasetToTin(lasdPath, tinPath, 'WINDOW_SIZE')
        print("Created TIN: ", tinPath)

    except: #failure
        print("Failed to create TIN. ArcPy error message:")
        print(arcpy.ExecuteError)
else:
    print("TIN file already exists at: ", tinPath)

TIN file already exists at:  C:\ArcGIS\Projects\Lab2_Part1\TIN_4342_12_06


In [38]:
#outputting PDF of DEM layer

demPDF = os.path.join(outputPath, 'DEM_4342_12_06.pdf') #where i will save my .pdf
demLayer = map.listLayers('DEM_4342_12_06')[0] #define layer i want to display
dmap = project.createMap("DEM Map") #make new map
dmap.addLayer(map.listLayers('DEM_4342_12_06')[0])  #add DEM layer to new map
demLayer = dmap.listLayers('DEM_4342_12_06')[0] #reassign demLayer to make sure it is correct
dlayout = project.createLayout(8.5, 11, 'INCH') #make new layout

#make a map frame using polygon geometry with a 0.5 inch border
envCoordList = [[0.5, 10.5], [8, 10.5], [8, 0.5], [0.5, 0.5], [0.5, 10.5]]
envArray = arcpy.Array([arcpy.Point(*coords) for coords in envCoordList])
polygon = arcpy.Polygon(envArray)
map_frame = dlayout.createMapFrame(polygon, dmap)
map_frame.camera.setExtent(map_frame.getLayerExtent(demLayer, False, True)) #set map frame to layer extent

#export layout to PDF
dlayout.exportToPDF(
    out_pdf = demPDF,
    output_as_image = True, #needed to make the file size smaller so i turned the layer into a compressed image
    image_quality = 'NORMAL'
)

if os.path.exists(demPDF): #success
    print("Created PDF of DEM layer: ", demPDF)
else: #failure
    print("PDF creation failed.")

Created PDF of DEM layer:  C:\ArcGIS\Projects\Lab2_Part1\DEM_4342_12_06.pdf


In [39]:
#outputting PDF of TIN layer

tinPDF = os.path.join(outputPath, 'TIN_4342_12_06.pdf') #where i will save my .pdf
tinLayer = map.listLayers('TIN_4342_12_06')[0] #define layer i want to display
tmap = project.createMap("TIN Map") #make new map
tmap.addLayer(map.listLayers('TIN_4342_12_06')[0])  #add TIN layer to the new map
tinLayer = tmap.listLayers('TIN_4342_12_06')[0] #reassign tinLayer to make sure it is correct
tlayout = project.createLayout(8.5, 11, 'INCH') #make new layout


#make new map frame with polygon geometry and 0.5 inch border
envCoordList = [[0.5, 10.5], [8, 10.5], [8, 0.5], [0.5, 0.5], [0.5, 10.5]]
envArray = arcpy.Array([arcpy.Point(*coords) for coords in envCoordList])
polygon = arcpy.Polygon(envArray)
map_frame = tlayout.createMapFrame(polygon, tmap)
map_frame.camera.setExtent(map_frame.getLayerExtent(tinLayer, False, True))

#export layout to PDF
tlayout.exportToPDF(
    out_pdf = tinPDF,
    output_as_image = True, #needed to make the file size smaller so i turned the layer into a compressed image
    image_quality = 'NORMAL'
)

if os.path.exists(tinPDF): #success
    print("Created PDF of TIN layer: ", tinPDF)
else: #failure
    print("PDF creation failed.")

Created PDF of TIN layer:  C:\ArcGIS\Projects\Lab2_Part1\TIN_4342_12_06.pdf
